In [45]:
import itertools

import numpy as np
from pyomo.environ import *
from pyomo.environ import value as pyoval
from typing import Tuple, Union
from collections import defaultdict

## VERTEX ENUMERATION

delta = min delta^k

delta^k = max delta

s.t.    A (d, z, theta^k) <= b
        theta = theta_nominal +/- theta_bounds
        z, theta >= 0

In [48]:
def vertex_enumeration(A:np.ndarray, b:np.ndarray, theta_nominal:list, theta_bounds:list, design_vectors:(np.ndarray, list[np.ndarray])=None, z_bounds:list[Tuple]=None):
    """
    A(d, z, theta) <= b
    Args:
        A: 2-D numpy array of coefficients of design, control, and uncertain parameters A(d, z, theta)
        b: 1-D numpy array of coefficients of constants
        theta_nominal: List of nominal values of uncertain parameters
        theta_bounds: List of bounds for uncertain parameters
        design_vectors: List of 1-D numpy arrays of design variable values (or a single array)
        z_bounds: List of bounds for control parameters

    Returns:

    """
    vertex_list = list(itertools.product(['+','-'], repeat=len(theta_nominal)))
    
    if not (isinstance(theta_nominal, list) and isinstance(theta_bounds, list) and all(isinstance(t, tuple) and len(t)==2 and all(isinstance(x, (int, float)) for x in t) for t in theta_bounds)):
        raise ValueError('Please provide a list (of length num_theta) of tuples (of size 2) and containing only float or integer values!')
    elif len(theta_nominal) != len(theta_bounds):
        raise ValueError('Inconsistent input for nominal theta values and theta bounds')
    
    num_theta = len(theta_nominal)
    num_design = len(design_vectors) if isinstance(design_vectors, list) else design_vectors.shape[0] if isinstance(design_vectors, np.ndarray) else 0
    num_control = A.shape[1] - num_theta - num_design
    
    if num_control < 0:
        raise ValueError('Inconsistent coefficients input')
        
    if z_bounds:        
        if isinstance(z_bounds, list) and len(z_bounds) == num_control and all(isinstance(t, tuple) and len(t)==2 and all(isinstance(x, (int, float)) for x in t) for t in z_bounds):
            print('z_bounds: ', z_bounds)
        else:
            raise ValueError('Please provide a list (of length num_control) of tuples (of size 2) and containing only float or integer values!')
        
    def evaluate_delta_fixed_design(z_b:list[Tuple]=None, design_vector:np.ndarray=None):
        model_min = None
        delta_min = None
        active_set_list = None
        vertex_min = None
        
        for vertex in vertex_list:
            m = ConcreteModel()
            m.Ai = RangeSet(A.shape[0])
            m.Aj = RangeSet(A.shape[1])
            m.i = RangeSet(num_design + num_control + num_theta)
    
            m.var = Var(m.i, within=NonNegativeReals) # structure (d,z,theta)
            m.delta = Var(within=NonNegativeReals)
            
            # system constraints: A(d,z,theta) <= b
            def Ax_leq_b_rule(instance, i):
                return sum(A[i-1, j-1]*instance.var[j] for j in instance.Aj) <= b[i-1]
            m.Ax_leq_b = Constraint(m.Ai, rule=Ax_leq_b_rule)
            
            # Add constraint for fixing first vars to the design value: d = dE
            if isinstance(design_vector, np.ndarray):
                m.d = RangeSet(num_design)
                def fix_design_rule(instance, d):
                    return instance.var[d] == design_vector[d-1]
                m.d_eq = Constraint(m.d, rule=fix_design_rule)
                
            if z_b:
                m.z = RangeSet(num_control)
                def z_LB_rule(instance, z):
                    return z_bounds[z-1][0] <= instance.var[num_design+z]
                m.z_lb = Constraint(m.z, rule=z_LB_rule)
                
                def z_UB_rule(instance, z):
                    return instance.var[num_design+z] <= z_bounds[z-1][1]
                m.z_ub = Constraint(m.z, rule=z_UB_rule)
            
            # theta constraints: theta = theta_nominal +/- theta_bounds * delta
            m.constraints = ConstraintList()
            theta_index = num_design + num_control + 1
            for i in range(len(theta_nominal)):
                if vertex[i]=='+':
                    expr = m.var[theta_index] == theta_nominal[i] + theta_bounds[i][1]*m.delta
                else:
                    expr = m.var[theta_index] == theta_nominal[i] - theta_bounds[i][0]*m.delta
                m.constraints.add(expr)
                theta_index+=1
            
            # Objective
            m.objective = Objective(expr=m.delta, sense=maximize)
            solver = SolverFactory('gurobi')
            solver.solve(m)
    
            if delta_min is None or pyoval(m.delta) < delta_min:
                delta_min = pyoval(m.delta)
                active_set_list = [f'f{c}' for c in m.Ax_leq_b if round(pyoval(m.Ax_leq_b[c].body),5) == round(pyoval(m.Ax_leq_b[c].upper),5)]
                model_min = m
                vertex_min = vertex
        
        return model_min, active_set_list, vertex_min, delta_min
    
    if isinstance(design_vectors, list):
        model_min, active_set_min, vertex_min, delta_min = [defaultdict(dict) for _ in range(4)]
        for i in range(len(design_vectors)):
            model_min[i], active_set_min[i], vertex_min[i], delta_min[i] = evaluate_delta_fixed_design(z_b=z_bounds, design_vector=design_vectors[i])
        return model_min, active_set_min, vertex_min, delta_min
    elif isinstance(design_vectors, np.ndarray):
        return evaluate_delta_fixed_design(z_b=z_bounds, design_vector=design_vectors)
    else:
        return evaluate_delta_fixed_design(z_b=z_bounds)

theta_nominal = [620, 388, 583, 313]
theta_bounds = [(10, 10), (10, 10), (10, 10), (10, 10)]

A = np.array([
    [-2/3, 0, 1, 0, 0],
    [-0.5, -0.75, -1, -1, 0],
    [1, -1.5, -2, -1, -2],
    [1, -1.5, -2, -1, 0],
    [-1, 1.5, 2, 1, 3]
])

b = np.array([350, -1388.5, -2830, -2044, 3153])

d0 = np.array([0.1, 0.2, 0.3, 0.4, 0.5])

model_VE2, active_set_list2, vertex2, delta2 = vertex_enumeration(A=A, b=b, theta_nominal=theta_nominal, theta_bounds=theta_bounds)

print(f'Flexibility index calculated through vertex enumeration: {delta2}')
print(f'Active set for vertex {vertex2}: {active_set_list2}')

Flexibility index calculated through vertex enumeration: 0.5600000000000023
Active set for vertex ('-', '-', '-', '-'): ['f1', 'f3']


## ACTIVE SET STRATEGY

In [49]:
def active_set_method(A:np.ndarray, b:np.ndarray, theta_nominal:list, theta_bounds:list, design_vectors:(np.ndarray, list[np.ndarray])=None, z_bounds:list[Tuple]=None):
    if not (isinstance(theta_nominal, list) and isinstance(theta_bounds, list) and all(isinstance(t, tuple) and len(t)==2 and all(isinstance(x, (int, float)) for x in t) for t in theta_bounds)):
        raise ValueError('Please provide a list (of length num_theta) of tuples (of size 2) and containing only float or integer values!')
    elif len(theta_nominal) != len(theta_bounds):
        raise ValueError('Inconsistent input for nominal theta values and theta bounds')
    
    num_theta = len(theta_nominal)
    num_design = len(design_vectors) if isinstance(design_vectors, list) else design_vectors.shape[0] if isinstance(design_vectors, np.ndarray) else 0
    num_control = A.shape[1] - num_theta - num_design
    
    if num_control < 0:
        raise ValueError('Inconsistent coefficients input')
        
    if z_bounds:        
        if isinstance(z_bounds, list) and len(z_bounds) == num_control and all(isinstance(t, tuple) and len(t)==2 and all(isinstance(x, (int, float)) for x in t) for t in z_bounds):
            print('z_bounds: ', z_bounds)
        else:
            raise ValueError('Please provide a list (of length num_control) of tuples (of size 2) and containing only float or integer values!')

In [50]:
# def active_set(theta_nominal:list, theta_bounds:list, bigM:float = 1e3):
#     m = ConcreteModel()
# 
#     m.j = Set(initialize=[1,2,3,4,5], ordered=True)
#     m.t = Set(initialize=[0,1,2,3], ordered=True)
# 
#     m.z = Var(within=NonNegativeReals)
#     m.T = Var(m.t, within=NonNegativeReals)
#     m.slack = Var(m.j, within=NonNegativeReals)
#     m.dual = Var(m.j, within=NonNegativeReals)
#     m.y = Var(m.j, within=Binary)
#     m.delta = Var(within=NonNegativeReals)
# 
#     m.f1 = Constraint(expr= -m.z/1.5 + m.T[1] -350 + m.slack[1] == 0)
#     m.f2 = Constraint(expr = -0.5*m.z - 0.75*m.T[0] - m.T[1] - m.T[2] + 1388.5 + m.slack[2] == 0)
#     m.f3 = Constraint(expr = m.z - 1.5*m.T[0] - 2*m.T[1] - m.T[2] - 2*m.T[3] + 2830 + m.slack[3] == 0)
#     m.f4 = Constraint(expr = m.z - 1.5*m.T[0] - 2*m.T[1] - m.T[2] + 2044 + m.slack[4] == 0)
#     m.f5 = Constraint(expr = -m.z + 1.5*m.T[0] + 2*m.T[1] + m.T[2] + 3*m.T[3] - 3153 + m.slack[5] == 0)
# 
#     m.dual_sum = Constraint(expr = sum(m.dual[i] for i in m.j) == 1)
#     m.df_dz = Constraint(expr = -m.dual[1]/1.5 + 0.5*m.dual[2] + m.dual[3] + m.dual[4] - m.dual[5] == 0)
# 
#     def complementary_rule(instance, j):
#         return instance.slack[j] - bigM*(1-instance.y[j]) <= 0
#     m.complementary = Constraint(m.j, rule=complementary_rule)
# 
#     def dual_UB_rule(instance, j):
#         return instance.dual[j] - instance.y[j] <= 0
#     m.dual_UB = Constraint(m.j, rule=dual_UB_rule)
# 
#     m.max_active_set = Constraint(expr = sum(m.y[j] for j in m.j) == len(list(m.z.keys())) + 1)
# 
#     def T_LB_rule(instance, t):
#         return theta_nominal[t] - m.delta * theta_bounds[t][0] - instance.T[t] <= 0
#     m.T_LB = Constraint(m.t, rule=T_LB_rule)
# 
#     def T_UB_rule(instance, t):
#         return instance.T[t] - theta_nominal[t] + m.delta * theta_bounds[t][1] <= 0
#     m.T_UB = Constraint(m.t, rule=T_UB_rule)
# 
#     m.objective = Objective(expr=m.delta, sense=minimize)
# 
#     solver = SolverFactory('gurobi')
#     solver.solve(m)
# 
#     active_set_list = [getattr(m, f'f{j}').name for j in m.j if pyoval(m.dual[j])]
# 
#     return m, active_set_list, pyoval(m.delta)
# 
# theta_nominal = [620, 388, 583, 313]
# theta_bounds = [(10, 10), (10, 10), (10, 10), (10, 10)]
# 
# model_AS, active_set_list, flexibility_index = active_set(theta_nominal=theta_nominal, theta_bounds=theta_bounds, bigM=700)
# 
# print(f'Flexibility index calculated through active set strategy: {flexibility_index}')
# print(f'Active set: {active_set_list}')